In [1]:
import pandas as pd
import numpy as np
import time, os
from scipy import stats

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
KERAS_BACKEND="tensorflow"

import tensorflow.keras.backend as backend
import tensorflow.keras.layers as layers

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow_addons.activations import mish

In [3]:
FX = np.load(os.getcwd()+'/pythonGame/saves/FX.npy')
FY = np.load(os.getcwd()+'/pythonGame/saves/FY.npy')

In [4]:
FX.shape,FY.shape

((300, 26), (300, 15))

In [5]:
def custom_loss(y_true, y_pred):    
    tracks = categorical_crossentropy(y_true[:,:10],y_pred[:,:10])
    colors = categorical_crossentropy(y_true[:,10:12],y_pred[:,10:12])
    actions = categorical_crossentropy(y_true[:,12:14],y_pred[:,12:14])
    
    combined = tracks + colors + actions
    return combined * (y_true[:,-1] + 1) # multiplied by pointadv in order to give more weight to better plays

get_custom_objects().update({'custom_loss': custom_loss})

In [6]:
def create_model(summary=True):
    data_in = layers.Input(shape=(26,))
    
    X = layers.Dense(39, activation=mish)(data_in)
    X = layers.Dense(52, activation=mish)(X) 
#     X = layers.Dense(52, activation=mish)(X) # Trying just two layers to start
    
    Tracks = layers.Dense(10, activation='softmax')(X)
    Colors = layers.Dense(2, activation='softmax')(X)
    Action = layers.Dense(2, activation='softmax')(X)
    Pointadv = layers.Dense(1, activation='relu')(X) # dummy var for pointadv, not being trained on for now
    
    Out = layers.Concatenate()([Tracks,Colors,Action,Pointadv])
    
    model = Model(data_in,Out)    
    model.compile(optimizer='adamax', loss=custom_loss)
    
    
    if summary: model.summary()
    return model

In [11]:
model = create_model()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 39)           1053        input_2[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 52)           2080        dense_6[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 10)           530         dense_7[0][0]                    
_______________________________________________________________________________________

In [12]:
np.random.seed(42)
p = np.random.permutation(len(FX))
TX, TY = FX[p[:int(len(FX) * 0.8)]], FY[p[:int(len(FX) * 0.8)]]
VX, VY = FX[p[int(len(FX) * 0.8):]], FY[p[int(len(FX) * 0.8):]]
# splitting data randomly into training and valdiation, arguably should be split by game instead of rows
# more games can be generated later for a better out of sample test.

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

In [14]:
model.fit(TX,TY, epochs=100, batch_size=32, verbose=1, validation_data=(VX,VY), shuffle=True,
         callbacks=[early_stopping])

Epoch 1/100
8/8 [==============================] - 0s 14ms/step - loss: 10.1473 - val_loss: 6.8108
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 7.4474 - val_loss: 6.1035
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 6.9591 - val_loss: 5.8154
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 6.4874 - val_loss: 5.5709
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 6.1482 - val_loss: 5.4661
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 5.9636 - val_loss: 5.3888
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 5.7900 - val_loss: 5.2819
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 5.6332 - val_loss: 5.1973
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 5.5044 - val_loss: 5.1373
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 5.3993 - val_loss: 5.0999
Epoch 11/100
8/8 [=========

In [15]:
model.save(os.getcwd()+'/pythonGame/saves/NeuralNetv1.h5')

In [16]:
# Checking to make sure the model can be loaded with the environment as defined by get_custom_objects.
# I have had one too many models be saved with poorly defined custom objects that refuse to be loaded.
NeuralNet = load_model(os.getcwd()+'/pythonGame/saves/NeuralNetv1.h5', custom_objects=get_custom_objects())

In [17]:
NeuralNet.predict(FX[:60])

array([[1.12667792e-02, 4.35592607e-03, 6.09135926e-01, 1.32182594e-02,
        2.88400110e-02, 6.88773021e-02, 4.01109597e-03, 5.06365579e-03,
        2.00527869e-02, 2.35178262e-01, 2.49357715e-01, 7.50642300e-01,
        9.75386858e-01, 2.46131569e-02, 1.33660603e+00],
       [9.67536122e-03, 3.32076009e-03, 5.67116439e-01, 7.60110375e-03,
        2.06705835e-02, 4.29567173e-02, 2.35968712e-03, 2.61116447e-03,
        1.45739419e-02, 3.29114228e-01, 1.37725100e-01, 8.62274945e-01,
        9.25840437e-01, 7.41595253e-02, 1.10547638e+00],
       [2.74418760e-02, 1.72614101e-02, 4.85348940e-01, 2.20448356e-02,
        4.21022959e-02, 8.92833546e-02, 1.07281860e-02, 2.16173325e-02,
        5.79973385e-02, 2.26174384e-01, 6.19506061e-01, 3.80493909e-01,
        9.19597089e-01, 8.04029256e-02, 1.39351916e+00],
       [6.68432331e-03, 2.02818587e-03, 4.41816717e-01, 2.72389594e-03,
        5.48959300e-02, 4.20270935e-02, 1.20628416e-03, 1.41166232e-03,
        2.55750818e-03, 4.44648355e-0